In [1]:
import requests, json
from collections import OrderedDict
from operator import getitem
import time
import pandas as pd
from web3 import Web3
from ens import ENS
import warnings

In [2]:
#ignore by message
warnings.filterwarnings("ignore", message="The log with transaction hash")

In [3]:
#RPC Calls
amberdata_rpc_url = "https://rpc.web3api.io/api/v2?x-api-key=YOUR_API_KEY"
w3 = Web3(Web3.HTTPProvider(amberdata_rpc_url))
ns = ENS.fromWeb3(w3)
# Print if web3 is successfully connected
print(w3.isConnected())

True


In [4]:
headers = {
    "accept": "application/json",
    "x-amberdata-blockchain-id": "ethereum-mainnet",
    "x-api-key": "YOUR_API_KEY"
}

In [5]:
#Empyt dict for storing returned data. 
collection = {}
id = 0

#Token Contract Address & Proxy Address need both... One for fetching ABI, the other for the delegated Calls. 
implemenationAddress = Web3.toChecksumAddress('0xF15a943787014461d94da08aD4040f79Cd7c124e')
proxyAddress = Web3.toChecksumAddress('0x830BD73E4184ceF73443C15111a1DF14e495C706')

#For Pagination
pagination = 0
paginationIncrement = 50
total_records_required = 500

In [6]:
#Fetch Contract ABI: 
abi_url = f'https://web3api.io/api/v2/contracts/{implemenationAddress}' 

In [7]:
#Fetching contract address ABI, formating and extracting ABI Only. Will use later for parsing logs. 
abi = requests.request("GET", url=abi_url, headers=headers)
abi = json.loads(abi.text)
abi = abi['payload']['abi']

In [8]:
#Create Web3 Contract Instance: 
myContract = w3.eth.contract(address=proxyAddress, abi=abi)

In [9]:
while pagination <= total_records_required:
    #Fetch Contract Events Logs. 
    url = f'https://web3api.io/api/v2/addresses/{proxyAddress}/logs?page={pagination}&size=50'
    response = requests.request("GET", url, headers=headers)
    while response.status_code == 429:
        time.sleep(1)
        response = requests.request("GET", url, params=querystring)
    batch_list = json.loads(response.text)["payload"]['records']
    for item in batch_list:
        try: 
            receipt = w3.eth.getTransactionReceipt(item["transactionHash"])
            logs = myContract.events.AuctionBid().processReceipt(receipt)
            id +=1
            collection[id] = {}
            collection[id]['nounId'] = logs[0]['args']['nounId']
            collection[id]['sender'] = logs[0]['args']['sender']
            collection[id]['sender_ENS'] = ns.name(logs[0]['args']['sender'])
            collection[id]['value'] = logs[0]['args']['value']
            collection[id]['value_eth'] = "{:.2f}".format(float(logs[0]['args']['value'])/1000000000000000000)
            collection[id]['extended'] = logs[0]['args']['extended']
            collection[id]['blockNumber'] = logs[0]['blockNumber']
            collection[id]['timestamp'] = item["timestamp"]
            collection[id]['transactionHash'] =  logs[0]['transactionHash'].hex()
        except NameError:
            print(NameError)
        except:
            print("No Events at this TX")
    print('Finished page',pagination)
    pagination += paginationIncrement

    

/opt/homebrew/lib/python3.10/site-packages/ens/main.py:285: FutureWarning: The function signature for resolver() will change in v6 to accept `name` as a the positional argument, over `normal_name`, and the method will instead normalize the name internally. You may migrate to using `name` by passing it in as a keyword, e.g. resolver(name="ensname.eth").
  warnings.warn(


No Events at this TX
No Events at this TX
No Events at this TX
No Events at this TX
No Events at this TX
No Events at this TX
Finished page 0
No Events at this TX
No Events at this TX
No Events at this TX
No Events at this TX
No Events at this TX
No Events at this TX
No Events at this TX
No Events at this TX
No Events at this TX
No Events at this TX
Finished page 50
No Events at this TX
No Events at this TX
Finished page 100
No Events at this TX
No Events at this TX
No Events at this TX
No Events at this TX
Finished page 150
Finished page 200
Finished page 250
Finished page 300
Finished page 350
Finished page 400
Finished page 450
Finished page 500


In [11]:
df = pd.DataFrame.from_dict(collection,orient='index')

In [12]:
df

,nounId,sender,sender_ENS,value,value_eth,extended,blockNumber,timestamp,transactionHash
1,548,0x2cd121B6AF2685459E09aA94fa5D68e2156f4158,p3te.eth,33470000000000000000,33.47,False,16219926,1671467171000,0x005504d335d5e3cb28400f5aa15fd983f572f91817b4...
2,548,0x1D3077F87EE7b08217E22f65FBe014ed06E51ddf,litturnip.eth,32810000000000000000,32.81,False,16219869,1671466475000,0xab82d082fc3efdd8beb24974cdd00dcf7a1606614e8b...
3,548,0x2cd121B6AF2685459E09aA94fa5D68e2156f4158,p3te.eth,32160000000000000000,32.16,False,16219818,1671465863000,0x07bc577ed2066632b205ed52da4ade90b5f6018bbdd3...
4,548,0x1D3077F87EE7b08217E22f65FBe014ed06E51ddf,litturnip.eth,31520000000000000000,31.52,False,16219779,1671465395000,0x0df568be8a555929e23b1151fd6909d9bcba92ae6bc6...
5,548,0x2cd121B6AF2685459E09aA94fa5D68e2156f4158,p3te.eth,30900000000000000000,30.90,False,16219533,1671462431000,0x6cd46a151956a7428498105b4524e5d9f156c5964d2c...
...,...,...,...,...,...,...,...,...,...
196,52,0x6c0Cf880cB20EefabFB09341Fba9e2Bd29ad3DFA,bigint.eth,125000000000000000000,125.00,True,13293675,1632555967000,0x6cfd46e5c80a8a8c105d3eaafde964a5137c6891a4d8...
197,52,0x4d2036407a023ea482db7bA19092d8ee08852Ad1,None,121000000000000000000,121.00,True,13293652,1632555773000,0xfe3f7bb8b44733970e6fdd8c23e7b34b65fc4c163e0c...
198,52,0x4d2036407a023ea482db7bA19092d8ee08852Ad1,None,121000000000000000000,121.00,True,13293652,1632555773000,0xfe3f7bb8b44733970e6fdd8c23e7b34b65fc4c163e0c...
199,52,0x6c0Cf880cB20EefabFB09341Fba9e2Bd29ad3DFA,bigint.eth,117770000000000000000,117.77,True,13293643,1632555693000,0xe528bb968a843cfb451173cd97189a403e585c0034ca...
